In [2]:
import os
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [3]:
def get_all_korean():

    def nextKorLetterFrom(letter):
        lastLetterInt = 15572643
        if not letter:
            return '가'
        a = letter
        b = a.encode('utf8')
        c = int(b.hex(), 16)

        if c == lastLetterInt:
            return False

        d = hex(c + 1)
        e = bytearray.fromhex(d[2:])

        flag = True
        while flag:
            try:
                r = e.decode('utf-8')
                flag = False
            except UnicodeDecodeError:
                c = c+1
                d = hex(c)
                e = bytearray.fromhex(d[2:])
        return e.decode()

    returns = []
    flag = True
    k = ''
    while flag:
        k = nextKorLetterFrom(k)
        if k is False:
            flag = False
        else:
            returns.append(k)
    return returns

In [4]:
def korean2label(letter):
    ch1 = (ord(letter) - ord('가'))//588
    ch2 = ((ord(letter) - ord('가')) - (588*ch1)) // 28
    ch3 = (ord(letter) - ord('가')) - (588*ch1) - 28*ch2
    return np.array([ch1,ch2,ch3])

In [5]:
recieved_letters = "갊갸곁곬교궤껴꽤꾜끝높뉑늬닠닦덤돛돼땍뛰뜻라럽롵뤄많몇묟무벚붉뷔뺨뽱쁘삯셴숲쉽쎄쏟쑉씩얹없왼읊읨잃쟨점좋줬짚짧쫄쯷찧찾쳇춰츄치캐캠쾅쿵탤턔토튠특팼퍗폐푀퓸핥흡횃훨"

In [6]:
recieved_letters_labels = [korean2label(l) for l in recieved_letters]

In [7]:
ak = get_all_korean()

In [8]:
akl = [korean2label(k) for k in ak]

In [9]:
# recieved_letters에서 초중종성중 최소 하나가 같은 글자 + 초중종성중 둘이 같은 글자

In [10]:
akmap = defaultdict(set)

In [11]:
pbar = tqdm(ak)
for l in pbar:
    labels = korean2label(l)
    for rl, rlabel in zip(recieved_letters, recieved_letters_labels):
        if np.sum(labels==rlabel) >= 1:
            akmap[l].add(rl)
    for al,alabel in zip(ak,akl):
        if np.sum(labels==alabel) >= 2:
            akmap[l].add(al)
    pbar.set_postfix(a=l, m=akmap[l])

100%|██████████| 11172/11172 [09:41<00:00, 19.20it/s, a=힣, m={'닠', '낗', '앃', '힐', '힋', '힉', '씩', '팋', '밓', '딯', '핳', '헣', '힓', '휳', '휗', '힝', '짚', '깋', '힑', '잏', '핥', '헇', '킿', '훟', '힣', '훨', '홯', '횧', '힌', '싷', '힒', '힞', '햫', '힇', '좋', '히', '횃', '삫', '핗', '혛', '힕', '힛', '훻', '힟', '맇', '힊', '힘', '홓', '힜', '흏', '힏', '힗', '힖', '힠', '띻', '흡', '힚', '짛', '힔', '빟', '횋', '힡', '치', '혷', '흫', '칳', '힢', '힍', '잃', '훃', '힎', '찧', '닣', '헿', '힙', '햏'}]                                                                                                                                             


In [12]:
fd = "/home/jupyter/ai_font/data/pickle"
os.makedirs(fd, exist_ok=True)

In [13]:
import pickle
with open(f"{fd}/letter_mapper.pickle", "wb") as f:
    pickle.dump(akmap, f)